In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.core.debugger import set_trace

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Specifically, we want to know what the literature reports about:

* Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time.
* Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged.
* Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over.
* Evidence of whether farmers are infected, and whether farmers could have played a role in the origin.
* Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia.
* Experimental infections to test host range for this pathogen.
* Animal host(s) and any evidence of continued spill-over to humans
* Socioeconomic and behavioral risk factors for this spill-over
* Sustainable risk reduction strategies

In [ ]:
#create mapping of file


map2file = dict()
_ignorefile = set(['COVID.DATA.LIC.AGMT.pdf', 'json_schema.txt', 'metadata.csv', 'metadata.readme'])
for dirname, _, filenames in os.walk('/kaggle/input/CORD-19-research-challenge'):
    for filename in filenames:
        if filename in _ignorefile:
            continue
            
        name = filename.split('.')
        if len(name) > 1 and name[1] == 'json':
            
            map2file[name[0]] = os.path.join(dirname, filename)


In [ ]:
exfile = pd.read_csv('/kaggle/input/new-labels-example/0_complete.csv')

In [ ]:

temp = exfile[exfile['_id'] == '0e38333bff68345492526fd39b70d1b18969cb83']
temp.head()


In [ ]:
class Remapit:
    def __init__(self,data_dir,label_file):
        self.paperids = list(set(label_file['_id'].values))
        self.map2file = self.create_map2file(data_dir)
        self.label_file = label_file
    def create_map2file(self,data_dir):
        map2file = dict()
        _ignorefile = set(['COVID.DATA.LIC.AGMT.pdf', 'json_schema.txt', 'metadata.csv', 'metadata.readme'])
        for dirname, _, filenames in os.walk('/kaggle/input/CORD-19-research-challenge'):
            for filename in filenames:
                
                #ignore files that are not research papers
                if filename in _ignorefile:
                    continue
                    
                #only include json files (research paper)
                name = filename.split('.')
                if len(name) > 1 and name[1] == 'json':

                    map2file[name[0]] = os.path.join(dirname, filename)
                    
        return map2file
        
    
    def process_pls(self):
        
        for p_id in self.paperids:
            print('Processing {}'.format(p_id))
            #get all rows relevant to current paper
            subdata = self.label_file[self.label_file['_id'] == p_id]
            #print(subdata.head())
            file_location = self.map2file[p_id]
            with open(file_location) as jsfile:
                paperjs = json.load(jsfile) # <- the paper from CORD19
                
                self.process_again(paperjs['body_text'],subdata)
                

    
    def process_again(self,paper_txt,sub_data):
        
        sec = 0
        run_len = 0 
        to_insert = []



        for i,dat in sub_data.iterrows():
            #get length of section at current index
                en_len = run_len + len(paper_txt[sec]['text'])
                if dat['startChar'] <= sen_len and dat['endChar'] <= sen_len:
                    #body of text is completely inside this section (both start and end is inside)

                    sub_data.at[i,'section'] = sec
                    d_case = 0
                elif dat['startChar'] <= sen_len and dat['endChar'] > sen_len:
                    #transitioning to new section (start is in section, end is in next section)

                    sub_data.at[i,'section'] = sec

                    #set_trace()
                    #replicate rows which are part of 2 sections
                    try:
                        temp = list(sub_data.loc[i].values)
                    except:
                        set_trace()
                    #print(len(temp))
                    #modify section at position 2
                    temp[2] = sec+1


                    to_insert.append(temp)

                    
                    run_len +=  len(paper_txt[sec]['text'])
                    sec+=1
                    d_case=1

                elif dat['startChar'] > sen_len and dat['endChar'] > run_len:
                    #next section

                    run_len +=  len(paper_txt[sec]['text'])
                    sub_data.at[i,'section'] = sec+1
                    sec+=1
                    d_case = 2
                print(d_case)
            except BaseException as e:
   
                set_trace()
                print(e)
        
        
        #add rows
        b = pd.DataFrame(to_insert,columns=sub_data.columns)
        
        pd.concat([sub_data,b])
        

In [ ]:
label_file = pd.read_csv('/kaggle/input/new-labels-example/0_complete.csv')

rmp = Remapit('/kaggle/input/CORD-19-research-challenge',label_file)

In [ ]:
print(len(rmp.label_file))


In [ ]:
rmp.process_pls()

In [ ]:
rmp.

In [ ]:
temp = rmp.label_file.iloc[2553].values

In [ ]:
temp[2]